In [45]:
import rasterio
import numpy as np
import pandas as pd
import os
import shelve

In [56]:
def read_image_data(image_dir='2014/images/',
                    mask_dir='2014/masks/',
                    table_dir='2014/tables/LC8_SR.csv', 
                    shelve_dir=None):
    if shelve_dir is None:
        combined = {} # write to file instead
        ds = {}
    else:
        combined = shelve.open(shelve_dir + 'combined')
        ds = shelve.open(shelve_dir + 'ds')
    for fn in os.listdir(image_dir):
        arr_img = rasterio.open(image_dir + fn).read()
        arr_msk = rasterio.open(mask_dir + fn).read()
        combined[fn.split('.')[0]] = np.concatenate((arr_img, arr_msk), axis=0)
    table = pd.read_csv(table_dir)
    time_start = table[['system:index', 'system:time_start']]
    
    for k, v in combined.items():
        ts = time_start[time_start['system:index'] == k]['system:time_start'].iloc[0]
        ds[str(ts)] = v
    if shelve_dir is not None:
        combined.close()
    return ds

In [57]:
dat = read_image_data(image_dir='area1/2014/images/',
                     mask_dir='area1/2014/masks/',
                     table_dir='area1/2014/tables/LC8_SR.csv',
                       shelve_dir='models/area1/shelves/')

In [64]:
def get_boolean_mask(image, level=1):
    cfmask = image[3, :, :]
    cfmask_conf = image[4, :, :]
    return ((cfmask == 0) | (cfmask == 1)) & (cfmask_conf <= level)


def zigzag_integer_pairs(max_x, max_y):
    total = 0
    x = 0
    while total <= max_x + max_y:
        if total - x <= max_y:
            yield (x, total - x)
        if x <= min(max_x - 1, total - 1):
            x += 1
        else:
            total += 1
            x = 0

In [68]:
ks = list(dat.keys())

In [71]:
keys_as_int = [int(k) for k in ks]

In [72]:
keys_as_int

[1393632987225,
 1394410211566,
 1395015377948,
 1395792595786,
 1397780144150,
 1398557367125,
 1399162529511,
 1399939752769,
 1400544919916,
 1401322152255,
 1401927327676,
 1402704560525,
 1404086963402,
 1404692138926,
 1405469370600,
 1406074541664,
 1406851776359,
 1408234182394,
 1408839353835,
 1409616584607,
 1410221757598,
 1410998985984,
 1412381389175,
 1412986561487,
 1413763792495,
 1414368960709,
 1415146191267,
 1415751363029,
 1416528587183,
 1417133760843,
 1417910986952,
 1418516154655,
 1419293385395,
 1419898551514,
 1420675781417,
 1421280950873,
 1422058174921,
 1422663346261]